In [14]:

import numpy as np
import math
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister,Aer, transpile, execute
from qiskit.visualization import plot_histogram

In [15]:
def create_lists(clause):
    pos_var = []
    neg_var = []
    none = []
    for index,item in enumerate(clause):
        if item>0:
            pos_var.append(index)
        elif item<0:
            neg_var.append(index)
        else:
             none.append(index)
    return pos_var,neg_var,none

In [16]:
def encode_clause(clause,circuit,qreg):
    
    num_of_variables = len(clause)
    positive_var,negative_var,either_var = create_lists(clause)
    
    #Building the oracle
    
    #negating the variables
    if len(negative_var)!=0:
        circuit.x(qreg[i] for i in negative_var)
        circuit.barrier()

    #Logical OR operation
    circuit.x([qreg[i] for i in (positive_var+negative_var)])
    #qc.mct([qreg[i] for i in range(num_of_variables)],qr[[-1]])
    circuit.mct([qreg[i] for i in (positive_var+negative_var)],qreg[-1])
    circuit.x([qreg[i] for i in (positive_var+negative_var)])
    circuit.barrier()

    #Bringing the negated variables to original form
    if len(negative_var)!=0:
        circuit.x(qreg[i] for i in negative_var)
        circuit.barrier()
    
    return circuit

In [17]:
def diffusion_circ(list):
    num_of_variables = len(list)
    
    qr = QuantumRegister(num_of_variables+1)
    cr = ClassicalRegister(num_of_variables+1)
    qc = QuantumCircuit(qr,cr)
    
    qc.h(qr[:])
    qc.x(qr[:])
    qc.h(qr[-1])
    qc.mct([qr[i] for i in range(num_of_variables)],qr[[-1]])
    qc.h(qr[-1])
    qc.x(qr[:])
    qc.h(qr[:])
    
    return qc

### 3-variable 3SAT problem

In [18]:
def mct_3(circuit,clause,aux,f_in,f_out):
    # circuit.ccx(control_qbits[0],control_qbits[1],aux[0])
    # circuit.ccx(control_qbits[2],aux[0],f_out[0])
    # circuit.ccx(control_qbits[0],control_qbits[1],aux[0])
    control_qbits = [i for i,val in enumerate(clause) if val!=0]
    if len(control_qbits)==3:
        circuit.ccx(f_in[control_qbits[0]],f_in[control_qbits[1]],aux[0])
        circuit.ccx(f_in[control_qbits[2]],aux[0],f_out[0])
        circuit.ccx(f_in[control_qbits[0]],f_in[control_qbits[1]],aux[0])
    elif len(control_qbits)==2:
        circuit.ccx(f_in[control_qbits[0]],f_in[control_qbits[1]],f_out[0])
    elif len(control_qbits)==1:
        circuit.cx(f_in[control_qbits[0]],f_out[0])
    return circuit

def oracle(circuit,f_in,f_out,aux,n,sat_formula):
    
    for clause in sat_formula:    
        for i in range(n):
            #Encoding positive variables
            if clause[i]>0:
                circuit.x(f_in[i])
        mct_3(circuit,clause,aux,f_in,f_out)
        for i in range(len(clause)):
            #Encoding positive variables
            if clause[i]>0:
                circuit.x(f_in[i])
        circuit.barrier()
        
    return circuit  

def diffusion_circ3(circuit,f_in):
    circuit.h(f_in[:])
    circuit.x(f_in[:])
    circuit.h(f_in[-1])
    circuit.ccx(f_in[0],f_in[1],f_in[-1])
    circuit.h(f_in[-1])
    circuit.x(f_in[:])
    circuit.h(f_in[:])
    circuit.barrier()
    
    return circuit

In [19]:
def sat3_var3(sat_formula):
    n = max(len(clause) for clause in sat_formula)
    print(n)
    f_in = QuantumRegister(n)
    f_out = QuantumRegister(1)
    aux = QuantumRegister(n-2)
    cr = ClassicalRegister(n)
    circuit = QuantumCircuit(f_in,f_out,aux,cr)

    #Superposition of input states
    for i in range(n):
        circuit.h(f_in[i])
    # circuit.x(aux[1])
    # circuit.h(aux[1])
    #initializing f_out qubit with state |1>
    circuit.x(f_out[0])
    circuit.barrier()
    
    #oracle
    oracle(circuit,f_in,f_out,aux,n,sat_formula)
    circuit.z(f_out[0])
    # circuit.cx(f_out[0],aux[1])
    circuit.barrier()
    #diffusion operator
    diffusion_circ3(circuit,f_in)
    #phase kickback

    circuit.measure(f_in[:],cr[:])
    
    return circuit
        